In [17]:
from sklearn import datasets, tree
# from id3 import Id3Estimator, export_text
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Load the breast cancer dataset
dataset = datasets.load_breast_cancer()

In [18]:
# Parse data into training and testing
data_size = len(dataset.data)
training_size = round(data_size * 0.8)

training_data = dataset.data[:training_size]
testing_data = dataset.data[training_size:data_size]

training_label = dataset.target[:training_size]
testing_label = dataset.target[training_size:data_size]

## Decision Tree Classifier

In [19]:
# Train model using decision tree
tree_decision = tree.DecisionTreeClassifier(random_state=0)
tree_decision = tree_decision.fit(training_data, training_label)
result = tree.export_text(tree_decision, feature_names=dataset['feature_names'].tolist())
print(result)

|--- worst perimeter <= 106.05
|   |--- worst smoothness <= 0.18
|   |   |--- worst concave points <= 0.16
|   |   |   |--- worst fractal dimension <= 0.06
|   |   |   |   |--- class: 0
|   |   |   |--- worst fractal dimension >  0.06
|   |   |   |   |--- worst texture <= 30.15
|   |   |   |   |   |--- area error <= 48.98
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- area error >  48.98
|   |   |   |   |   |   |--- mean compactness <= 0.06
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- mean compactness >  0.06
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- worst texture >  30.15
|   |   |   |   |   |--- mean fractal dimension <= 0.06
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- mean fractal dimension >  0.06
|   |   |   |   |   |   |--- smoothness error <= 0.01
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- smoothness error >  0.01
|   |   |   |   |   |   |   |--- mean radius <= 12.38
|  

In [20]:
# Predict using the model
tree_results = tree_decision.predict(testing_data)

# Evaluate the model
print(accuracy_score(testing_label, tree_results))
print(f1_score(testing_label, tree_results, average='macro'))

0.8421052631578947
0.8044969512195121


## Id3 Estimator

In [21]:
# id3_estimator = Id3Estimator()
# id3_estimator = id3_estimator.fit(training_data, training_label)
# id3_tree = export_text(id3_estimator.tree_, dataset['feature_names'])
# print(id3_tree)

## K-Means

In [22]:
# Run K-Means clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(training_data)
kmeans_results = kmeans.predict(testing_data)

# Evaluate the model
print(accuracy_score(testing_label, kmeans_results))
print(f1_score(testing_label, kmeans_results, average='macro'))

0.9122807017543859
0.8591897233201581


## Neural Network

In [25]:
neural_network = MLPClassifier(max_iter=300).fit(training_data, training_label)
neural_network_results = neural_network.predict(testing_data)
# neural_network.score(testing_data, testing_label)

# Evaluate the model
print(accuracy_score(testing_label, neural_network_results))
print(f1_score(testing_label, neural_network_results, average='macro'))

0.9122807017543859
0.8843813387423934
